## Group 3 Mobile Robotics Report

**students**: Marc Nassif,  Luca Simon Engel,  Giuseppe De Carlo,  Giada Ehrlich



## Table of contents
1. [Introduction](#1-introduction)
2. [Implementation](#2-implementation)
      - 2.1 [Vision](#3-global-navigation)
         -  [Implementation Details](#31-implementation-details)
         -  [Functions](#32-functions)
      - 2.2 [Global Navigation](#4-vision)
         -  [Implementation Details](#41-implementation-details)
         -  [Functions](#42-functions)
      - 2.3 [Motion Control](#5-motion-control)
         -  [Implementation Details](#51-implementation-details)
         -  [Functions](#52-functions)
      - 2.4 [Local Navigation](#6-local-navigation)
         -  [Implementation Details](#61-implementation-details)
         -  [Functions](#62-functions)
      - 2.5 [Kalman Filter](#7-kalman-filter)
         - [Implementation Details](#71-implementation-details)
         - [Functions](#72-functions)
3. [Results](#4-results)
   - 3.1 [Summary](#41-summary)
   - 3.2 [Videos](#42-video)
4. [Conclusion](#4-conclusion)
   - 4.1 [Summary](#41-summary)
   - 4.2 [References](#42-references)


   






    

## 1. Introduction




In mobile robotics, fusing computer vision and sensor-based feedback is essential to establish reliable global navigation algorithms, motion control strategies, and sensor-based local obstacle avoidance to enable robots to intelligently and reliably navigate through dynamic environments. This report describes our effort to develop such a solution to enable Thymio robots to traverse a terrain containing obstacles to reach a goal.

Our project integrates multiple technologies to improve the navigation precision. With the help of a video feed provided by a camera, computer vision is used to detect the map, the obstacles in the map, the Thymio robot, and the goal. Based on this information, a 2d grid of cells is constructed.

The global navigation aspect employs the Dijkstra algorithm to calculate the shortest path from the Thymio to the goal, accounting for dynamically changing events, such as kidnapping where the Thymio or the goal are displaced.

The motion control mechanism guided the Thymio along the calculated path which is discretized into intermediate points, facilitating a smooth and precise traversal.

For local obstacle avoidance, the Thymio relies on its proximity sensors, allowing it to detect and navigate around obstacles not identified by the computer vision part. This ensures robust navigation, even when visual data is limited.

The implementation of the Kalman filter for position and orientation estimation plays a pivotal role in this project. The filter combines information from the computer vision part and the wheel speeds of the Thymio to improve robustness of the navigation. The filter is even capable of functioning when the camera feed is obscured for a while, showing the robustness of our system.

This report details the design, implementation, and performance of our solution and shows how the combination of sensors and data can drastically improve the capabilities and precision in robotic navigation.

## 2. Implementation
   

### 2.2 Vision


Our vision system forms the baseline of the Thymio's understanding of the environment. Using the OpenCV library, the map is created in multiple steps, each of which extracting information from the camera feed, allowing the robot to identify markers, determining their orientation, and recognizing obstacles. In the following, our procedure will be shown with the example of the following image feed:

![Image Alt Text](./report_images/image_processing_images/simple_camera_feed.png)


##### Marker Detection
We used OpenCV's arUco marker detection algorithm to find the key elements and their orientation within the terrain, specifically, the Thymio robot, the goal, and the corners of the map. Based on this information, a transformation matrix is applied to the image to place the map corners into the corners of the image so that the map fills the entire video feed.

![Image Alt Text](./report_images/image_processing_images/processed_base_frame.png)

##### Grayscale Conversion and Thresholding
To simplify the object detection, we converted the camera feed to grayscale. As the objects are represented as black shaped, applying thresholding to the image allowed for easy extraction of their location in the environment.

![Image Alt Text](./report_images/image_processing_images/binary_img.png)

The resulting image still contains some noise due to the black thymio sensors being detected as obstacles. This is resolved in the next step. 

##### 2D Grid Representation
Based on the information obtained in the previous steps, a 2D grid of cells is created. Each cell stores information about its status, indicating whether it is free, occupied by the Thymio, the goal, or an obstacle. 

After conversion, the 2D grid looks as follows:

![Image Alt Text](./report_images/image_processing_images/grid_map_with_islands_not_increased_object_size.png)

Here, still, the map contains the noise from the black sensors of the Thymio. This is solved by removing all obstacles that are detected to be small islands. Additionally, to account for the width of the Thymio, obstacles are enlarged to ensure that the Thymio cannot drive too close to the obstacles. The resulting map looks as follows:

![Image Alt Text](./report_images/image_processing_images/grid_map.png)




##### Dynamic Updates
The grid is continuously updated with the positions of the Thymio and the goal within the map. These updates ensure that the Thymio can make informed decisions, such as path recomputation after kidnapping events or motion control adjustments.

This part lays the groundwork for successful navigation of the Thymio robot in the terrain.

In [1]:
#code: 


### 2.2 Global Navigation



Our global navigation system guides the Thymio robot throught the map by calculating the shortest path from its start location to the goal. This path is found with the Dijkstra algorithm. Similar to the vision, the navigation is also constructed in a multi-step approach.

##### Dijkstra Algorithm
The shortest path is calculated using the Dijkstra algorithm. The resulting path ensures that the Thymio can navigate through the environment while avoiding obstacles and ensures that it efficiently reaches the goal.

##### Path Discretization
To have the Thymio drive more smoothly, the computed path is discretized into a series of fewer cells. Each cell represents a direction change in the path. The Thymio receives these cells as intermediate goals allowing a more natural movement along the path and decreasing the number of sudden direction changes.

##### Integration with Vision System
Based on the information received from the vision system, the path is calculated and obstacles can be avoided. Additionally, kidnapping of the thymio and the goal can be handled. Whenever such a kidnapping occurs, the path is recomputed so that the goal can be reached.


The combination of the Dijkstra algorithm, path discretization, and intermediate goal navigation, ensures a smooth and fast traversal of the map, even accounting for sudden events such as kidnapping.

In [ ]:
# functions: 

### 2.3 Motion Control

details: 


In [ ]:
# functions: 


### 2.4 Local Navigation

details:

In [ ]:
# functions


### 2.5 Kalman Filter

Our Kalman filter implementation serves a critical role in the position and orientation estimation of our Thymio. It uses the KalmanFilter library of pythons filterpy package and combines the wheel motor speed of the Thymio with the information provided by the computer vision part to increase accuracy.

##### Wheel Speed Measurements
The wheel speed measurements are used for both estimating the position and orientation of the Thymio. To predict the new orientation, the angular velocity is calculated based on the difference of wheel speeds and a baseline minimum period for a 360° turn of the Thymio. Additionally, the wheel speeds are used to estimate the new position of the Thymio. 

##### Camera input
As the camera input already is provided in grid coordinates by the computer vision, there is no additional processing needed for it.

##### Prediction and Update Step
We use the wheel speed measurements to perform the prediction step of the Kalman filter and use the computer vision information in the update method. With the combination of these two informations, the Kalman filter provides an accurate estimation of the position and orientation of the Thymio robot.

 ##### Handling Orientation Jumps
As the provided orientation input from our computer vision module is a float between 0 and 2π, depending on the Thymio's orientation, there may be jumps in the estimated orientation from 0 to 2π and vice versa. This leads to large jumps in the overall estimation of the orientation. To combat that, we do not limit the predicted orientation to [0, 2π] but let it be any number and scale using the mod operation only when the navigation part accesses it. This dramatically improved the orientation estimation of our filter.

##### Adaptability to Vision Feed Cut-Off
Whenever the camera feed is unavailable, the Kalman filter continues to estimate the postition and orientation based solely on the predictions derived from the wheel speed measurements. This allows the Thymio to maintain knowledge of its position and orientation even when external measurements are unavailable

The Kalman filter's ability to combine information from wheel speed measurements and vision data drastically increases the robustness of our navigation and even allows for accurate driving when no computer vision data is available.

In [4]:
#Functions

### Results


video: 
